# paiza 練習問題で学ぶアルゴリズム
## [日別訪問者数の最大平均区間(large) (paizaランク A 相当)](https://paiza.jp/works/mondai/skillcheck_archive/max_range_large)を解いてみよう！

Bランクの練習問題はクリア出来ましたが、同じ内容のAランク問題では100点を取ることが出来ませんでした。  
違いを調べ、ソースコードを改善してAランク問題も攻略しましょう！

### 1. 計算量を調べてみよう

計算量とは、プログラムを実行した時の、大雑把な計算回数のことです。  
入力例1、入力例2で計算の動きを見てみましょう。

<br>
<div style="text-align: center;">
  入力例1 計算する区間(ログ: n = 5, キャンペーン: k = 3)
</div>

|入力例1|day1|day2|day3|day4|day5|
|:--:|:--:|:--:|:--:|:--:|:--:|
|訪問者数|1|2|3|2|1|
|キャンペーン候補1|1|2|3|||
|キャンペーン候補2||2|3|2||
|キャンペーン候補3|||3|2|1|

<br>
<div style="text-align: center;">
    キャンペーン候補区間数: 3、区間の要素数: 3、計算回数 3 × 3 = 9回
</div>
<br>

---

<div style="text-align: center;">
  入力例2 計算する区間(ログ: n = 10, キャンペーン: k = 2)
</div>

|入力例2|day1|day2|day3|day4|day5|day6|day7|day8|day9|day10|
|:--:|:--:|:--:|:--:|:--:|:--:|:--:|:--:|:--:|:--:|:--:|
|訪問者数|6|2|0|7|1|3|5|3|2|6|
|キャンペーン候補1|6|2|||||||||
|キャンペーン候補2||2|0||||||||
|キャンペーン候補3|||0|7|||||||
|キャンペーン候補4||||7|1||||||
|キャンペーン候補5|||||1|3|||||
|キャンペーン候補5||||||3|5||||
|キャンペーン候補5|||||||5|3|||
|キャンペーン候補5||||||||3|2||
|キャンペーン候補6|||||||||2|6|

<br>
<div style="text-align: center;">
    キャンペーン候補区間数: 6、区間の要素数: 2、計算回数 6 × 2 = 12回
</div>
<br>

#### 計算回数についてまとめる

- 計算回数 = (n - k + 1) * k

- n が一定の場合、k が n / 2 のとき、計算量が一番大きくなる
  - n=5, k=1  ->  (5 - 1 + 1) * 1 = 5
  - n=5, k=2  ->  (5 - 2 + 1) * 2 = 8
  - **n=5, k=3  ->  (5 - 3 + 1) * 3 = 9**
  - n=5, k=4  ->  (5 - 4 + 1) * 4 = 8
  - n=5, k=5  ->  (5 - 5 + 1) * 5 = 5

タイムオーバーのケース n = 300000、k = 150000 だと…

**n=300000, k=150000  ->  (300000 - 150000 + 1) * 150000 = 22,500,150,000（225億15万回!!）**  
超ざっくり、桁の部分だけを取り出すと 10^5 * 10^5 = 10^10 （10の10乗）オーダの計算だったということになります。

※ ちなみに判定では１６秒で打ち切られましたが、プログラムを最後まで実行すると３６秒かかりました。

### 2. Bランク問題とAランク問題の違いについて

Bランク問題では、ログデータの件数が最大1000件でしたが、Aランク問題では最大30万件となっています。  
データ件数が少ないうちは処理の流れが正しく書けていれば問題はないのですが、処理件数が多くなると、制限時間内で処理しきれないケースが出てきます。（paizaではタイムオーバー、AtCoderではTime Limit Exceededという失敗になります）  
これは、出題側が想定した計算量をオーバーしているということになりますので、そのへんを解決する工夫がAランク以上では求められます。

さて、計算数を減らす工夫をしなければいけないのですが、自力ではなかな思いつけないですよねぇ...（本番問題だと時間制限もあるし）  
そこで使われるのが、「代表的なアルゴリズム」の数々というわけです！  
「ユークリッドの互除法」とか、「エラトステネスの篩（ふるい）」など聞いたことないでしょうか？

Aランク以上では、そのような代表的な「アルゴリズム」を、問題に合わせて選択し、応用する能力が必要というわけです。  
数学で言えば公式を覚えて応用するようなイメージでしょうかね。

#### 代表的なアルゴリズム
- 貪欲法
- 分割統治法
- 動的計画法
- 幅優先探索
- 深さ優先探索


### 3. プログラムの改善を考える



---

#### タイムオーバーしたプログラム

```ruby
# データ入力
log_count, campaign_days = gets.split.map(&:to_i)
visitor_count = gets.split.map(&:to_i)

# 区間平均を求める
section_averages = []

# 【 n-k+1 のループ 】
(0..log_count - campaign_days).each do |idx|
    
  # 【 k のループ 】
  section_averages << visitor_count[idx..idx + campaign_days - 1].sum.fdiv(campaign_days)
end

# 【 ここから先は計算量の対象としません 】

# maxメソッドで、最大平均(max_average)を求める
max_average = section_averages.max
# countメソッドで、最大平均の数(キャンペーン候補)を求める
candidate_count = section_averages.count(max_average)
# find_indexメソッドで、キャンペーン最初の候補日を求める(index 0 が 1日目なので index + 1 とする)
first_day = section_averages.find_index {|average| average == max_average} + 1
# 結果を出力
puts "#{candidate_count} #{first_day}"
```

---

#### 改善したプログラム

```ruby
# データ入力
log_count, campaign_days = gets.split.map(&:to_i)
visitor_counts = gets.split.map(&:to_i)

# 初回のキャンペーン候補区間の合計を求める　【 k のループ 】
section_visitors = [visitor_counts[0..campaign_days - 1].sum]

# 次のキャンペーン候補区間からの処理 【 n - k のループ】
(1..log_count - campaign_days).each do |current_idx|
    
  # キャンペーン候補から除く訪問者数　【 1回 】
  remove_idx = current_idx - 1
  # キャンペーン候補日に加える訪問者数　【 1回 】
  insert_idx = current_idx + campaign_days - 1

  # 今回のキャンペーン区間の訪問者数を計算する　【 1回 】
  section_visitors << section_visitors[-1] - visitor_counts[remove_idx] + visitor_counts[insert_idx]
end

# 【 ここから先は計算量の対象としません 】

# maxメソッドで、最大来場者数を求める
max_visitors = section_visitors.max

# countメソッドで、最大来場者数だった日数(キャンペーン候補)を求める
candidate_count = section_visitors.count(max_visitors)
# find_indexメソッドで、キャンペーン最初の候補日を求める(index 0 が 1日目なので index + 1 とする)
first_day = section_visitors.find_index { |visitor_count| visitor_count == max_visitors } + 1

# 結果を返す
puts "#{candidate_count} #{first_day}"
```
---

ループ内でsumメソッドを使っていた部分を改善することで、計算量をかなり減らすことができました！

- 改善前の計算回数 = (n - k + 1) * k --> (300,000 - 150,000 + 1) * 150000 = 22,500,150,000
- 改善後の計算回数 = k + (n - k) * 3 --> 150,000 + (300,000 - 150,000) * 3 = 600,000


### 5. データ構造とデータ処理（アルゴリズム）をRubyで実装する

プログラミングしなくても、とりあえず解けちゃいました。

条件2の範囲で色々な数値が入りますので、それらに対応できるコードを書いていきましょう！

ここまでの考え方が間違っていなければ、どんなプログラミング言語でも実装出来るはず！！

### 6. 入力データ受け取り

2行でスペース区切りの文字列データが与えられますので受け取っていきます。

#### 1行目のデータ

1行目の入力データは  ```"5 3\n"``` です。

5 がログの件数、 3 がキャンペーン日数なので

```ruby
raw_input = gets.chomp.split(" ")  # raw_input = ["5", "3"]
log_count = raw_input[0].to_i         # log_count = 5
campaign_days = raw_input[1].to_i     # campaign_days = 3
```

これで、 ```log_count = 5```, ```campaign_days = 3``` が受け取れました。

(今回の場合、```chomp```, ```split``` の引数```(" ")``` は省略可能でしたが、基本形の説明のため入れています)

#### 2行目のデータ

2行目の入力データは  ```"1 2 3 2 1\n"``` です。これを配列にしたいので、

```ruby
visitor_count = gets.chomp.split(" ").map(&:to_i)
```

これで、```visitor_count = [1, 2, 3, 2, 1]``` が受け取れました。

省略出来るところを省略すればもう少し短く書けます。

```ruby
log_count, campaign_days = gets.split.map(&:to_i)  # log_count, campaign_days = [5, 3]
visitor_count = gets.split.map(&:to_i)             # visitor_count = [1, 2, 3, 2, 1]
```

動作確認してみます。（```gets``` メソッドの代わりにデータを直接入力します。）

In [4]:
# 省略なし

# getsメソッドの代わり
input_data1 = "5 3\n"
input_data2 = "1 2 3 2 1\n"

# データ入力
raw_input = input_data1.chomp.split(" ")
log_count = raw_input[0].to_i
campaign_days = raw_input[1].to_i
visitor_count = input_data2.split(" ").map(&:to_i)

puts "log_count = #{log_count}"
puts "campaign_days = #{campaign_days}"
puts "visitor_count = #{visitor_count}"

log_count = 5
campaign_days = 3
visitor_count = [1, 2, 3, 2, 1]


In [5]:
# 省略あり

# getsメソッドの代わり
input_data1 = "5 3\n"
input_data2 = "1 2 3 2 1\n"

# データ入力
log_count, campaign_days = input_data1.split.map(&:to_i)
visitor_count = input_data2.split.map(&:to_i)

puts "log_count = #{log_count}"
puts "campaign_days = #{campaign_days}"
puts "visitor_count = #{visitor_count}"

log_count = 5
campaign_days = 3
visitor_count = [1, 2, 3, 2, 1]


### 7. データ処理の実装

データ処理部分をコーディングし、問題文で提示されている入力値を色々変えてみて動作確認します。

(変数名が変だったりするのはゆるして… orz)

In [6]:
# getsメソッドの代わり
input_data1 = "5 3\n"
input_data2 = "1 2 3 2 1\n"

#input_data1 = "10 2\n"
#input_data2 = "6 2 0 7 1 3 5 3 2 6\n"

# データ入力
log_count, campaign_days = input_data1.split.map(&:to_i)
visitor_count = input_data2.split.map(&:to_i)

# 区間平均を求める
section_averages = []

(0..log_count - campaign_days).each do |idx|
  # fdiv メソッドを使うと浮動小数点の商が得られます。
  section_averages << visitor_count[idx..idx + campaign_days - 1].sum.fdiv(campaign_days)
  
  # こう書くと、剰余が切り捨てられてしまいます。
  # section_averages << visitor_count[idx..idx + campaign_days - 1].sum / campaign_days
end

# 最大平均を求める
max_average = section_averages.max
# 最大平均の数(キャンペーン候補)を求める
candidate_count = section_averages.count(max_average)
# キャンペーン最初の候補日を求める(index 0 が 1日目なので index + 1)
first_day = section_averages.find_index {|average| average == max_average} + 1
# 結果を出力
puts "#{candidate_count} #{first_day}"

# 結果確認
puts <<~"EOS"
--- input
ログの数       : #{log_count}
キャンペーン日数 : #{campaign_days}
来客数         : #{visitor_count}
--- output
区間平均人数    : #{section_averages}
最大平均       : #{max_average}
最大平均の数    : #{candidate_count}
最初の候補日    : #{first_day}
EOS

1 2
--- input
ログの数       : 5
キャンペーン日数 : 3
来客数         : [1, 2, 3, 2, 1]
--- output
区間平均人数    : [2.0, 2.3333333333333335, 2.0]
最大平均       : 2.3333333333333335
最大平均の数    : 1
最初の候補日    : 2



### 8. 回答を提出

```gets``` メソッドを使用するコードに戻し、不要部分を削除してコピーし、paizaの解答欄に提出（ペースト）します。

```ruby
# データ入力
log_count, campaign_days = gets.split.map(&:to_i)
visitor_count = gets.split.map(&:to_i)

# 区間平均を求める
section_averages = []

(0..log_count - campaign_days).each do |idx|
  section_averages << visitor_count[idx..idx + campaign_days - 1].sum.fdiv(campaign_days)
end

# 最大平均を求める
max_average = section_averages.max
# 最大平均の数(キャンペーン候補)を求める
candidate_count = section_averages.count(max_average)
# キャンペーン最初の候補日を求める(index 0 が 1日目なので index + 1)
first_day = section_averages.find_index {|average| average == max_average} + 1
# 結果を出力
puts "#{candidate_count} #{first_day}"
```

### 9. まとめ

以上が問題提出までのざっくりとした流れとなります。（本番のスキルチェックは提出までの時間による減点がありますが基本は一緒です。）

標準入力のデータは、今回は2行だったので```gets```で書きましたが、たくさん受け取る場合もあります。他の受け取り方については機会があればご紹介したいと思います。

### 10. おまけ

実はこの問題には[日別訪問者数の最大平均区間(large)](https://paiza.jp/works/mondai/skillcheck_archive/max_range_large)という続きがあります。

とはいっても、与えられるデータログの数が ```1 ≦ n ≦ 1,000``` から ```1 ≦ n ≦ 300,000``` に変わるだけです。

処理内容は一緒なので、このまま提出してみましょう！！

どうなりました？

---

この結果が次の発表のネタとなります。

今回の発表は以上です。ありがとうございました！

#### 参考文献:

- [paiza開発日誌 【累積和、しゃくとり法】初級者でも解るアルゴリズム図解](https://paiza.hatenablog.com/entry/2015/01/21/%E3%80%90%E7%B4%AF%E7%A9%8D%E5%92%8C%E3%80%81%E3%81%97%E3%82%83%E3%81%8F%E3%81%A8%E3%82%8A%E6%B3%95%E3%80%91%E5%88%9D%E7%B4%9A%E8%80%85%E3%81%A7%E3%82%82%E8%A7%A3%E3%82%8B%E3%82%A2%E3%83%AB%E3%82%B4)